### EMA imports

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from ema_workbench import (ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, 
Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging)

ema_logging.log_to_stderr(ema_logging.INFO)

### Define Mesa model

In [ ]:
def model_virus_on_network(num_nodes=1, 
                            avg_node_degree=1, 
                            initial_outbreak_size=1, 
                            virus_spread_chance=1, 
                            virus_check_frequency=1, 
                            recovery_chance=1, 
                            gain_resistance_chance=1,
                            steps=10):
    
    from virus_on_network import model
    
    virus_on_network = model.VirusOnNetwork(num_nodes=num_nodes, 
                                avg_node_degree=avg_node_degree, 
                                initial_outbreak_size=initial_outbreak_size, 
                                virus_spread_chance=virus_spread_chance, 
                                virus_check_frequency=virus_check_frequency, 
                                recovery_chance=recovery_chance, 
                                gain_resistance_chance=gain_resistance_chance)
    
    virus_on_network.run_model(steps)
    outcomes = virus_on_network.datacollector.get_model_vars_dataframe()
    
    return {'TIME':list(range(steps + 1)),
        "Infected": outcomes["Infected"].tolist(),
            "Susceptible": outcomes["Susceptible"].tolist(),
            "Resistant": outcomes["Resistant"].tolist()}
    

### Test Mesa model

In [ ]:
model_virus_on_network(num_nodes=20, 
                            avg_node_degree=3, 
                            initial_outbreak_size=1, 
                            virus_spread_chance=0.4, 
                            virus_check_frequency=0.2, 
                            recovery_chance=0.4, 
                            gain_resistance_chance=0.3,
                            steps=10)

### Design EMA experiments

In [ ]:
model = ReplicatorModel('virusnetwork', function=model_virus_on_network)

In [ ]:
model.uncertainties = [IntegerParameter("num_nodes", 10, 100),
                       IntegerParameter("avg_node_degree", 2, 8),
                        RealParameter("virus_spread_chance", 0.1, 1),
                        RealParameter("virus_check_frequency", 0.1, 1),
                        RealParameter("recovery_chance", 0.1, 1),
                        RealParameter("gain_resistance_chance", 0.1, 1)]

In [ ]:
model.constants = [Constant("initial_outbreak_size", 1),
                  Constant('steps', 50)]

In [ ]:
model.outcomes = [TimeSeriesOutcome('TIME'),
                TimeSeriesOutcome('Infected'),
                 TimeSeriesOutcome('Susceptible'),
                 TimeSeriesOutcome('Resistant')]

In [ ]:
model.replications = 10

### Perform EMA experiments

In [ ]:
results = perform_experiments(model, 100)

experiments, outcomes = results

### Process outcomes

In [ ]:
import random

In [ ]:
outcomes[random.choice(list(outcomes))].shape

In [ ]:
#get shape of every outcome (should all be identical?)
# for k in list(outcomes):
#     print(outcomes[k].shape)

In [ ]:
outcomes_2D = {key:np.mean(outcomes[key],axis=1) for key in outcomes.keys()}
results_2D = (experiments.copy(), outcomes_2D)

In [ ]:
outcomes_2D[random.choice(list(outcomes))].shape

### Visualize outcomes

In [ ]:
from ema_workbench.analysis.plotting import lines

In [ ]:
plt.rcParams['figure.figsize'] = [10, 12]

figure = lines(experiments, outcomes_2D) #show lines, and end state density
plt.show()